The tool decorator-A tool decorator that automates the process of converting a Python function into a tool definition suitable for the LLM's system prompt.

In [1]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool #the decorator
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [2]:
def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from the Hacker News.

    This function retrieves the top 'top_n' stories from the Hacker News API.
    Each story contains the title, URL, score, author and time of submission. The data is fetched from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): Number of stories to fetch.
    """

    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()

        top_story_ids = response.json()[:top_n]

        top_stories = []

        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()
            story_data = story_response.json()

            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })

        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []


1.fetch_top_hacker_news_stories(top_n:int): function that fetches data from the Hacker News APi.


->It uses the request library to make HTTP GET requests


->It includes robust error handling for network requests(requests.exceptions.RequestException)



3.hn_tool.name and json.loads(hn_tool.fn_signature):These lines demonstrate that the hn_tool object now holds the automatically generate metadata.




In [3]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'Show HN: Draw A Fish and watch it swim with the others',
  'url': 'https://drawafish.com'},
 {'title': 'Gemini 2.5 Deep Think',
  'url': 'https://blog.google/products/gemini/gemini-2-5-deep-think/'},
 {'title': 'Fast (2019)', 'url': 'https://patrickcollison.com/fast'},
 {'title': 'Show HN: Pontoon – Open-source customer data syncs',
  'url': 'https://github.com/pontoon-data/Pontoon'},
 {'title': 'Show HN: An interactive dashboard to explore NYC rentals data',
  'url': 'https://leaseswap.nyc/analytics'}]

In [4]:
hn_tool = tool(fetch_top_hacker_news_stories)

hn_tool = tool(fetch_top_hacker_news_stories):This line applies the tool decorator to the fetch_top_hacker_news_stories function.

In [5]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [6]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': "\n    Fetch the top stories from the Hacker News.\n\n    This function retrieves the top 'top_n' stories from the Hacker News API.\n    Each story contains the title, URL, score, author and time of submission. The data is fetched from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): Number of stories to fetch.\n    ",
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [9]:
tool_agent = ToolAgent(
    tools=[hn_tool],
    model="llama-3.3-70b-versatile"
)

1.tool_agent = ToolAgent(
    tools=[hn_tool],
    model="llama-3.3-70b-versatile"
):Initializes the ToolAgent by providing it with a list of available tools, responsible for constructing the system prompt with all tool definition, managing the chat history, and orchestrating the LLM calls and tool execution.

In [10]:
output = tool_agent.run(user_msg="Tell me your name")

In [11]:
print(output)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [12]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "Show HN: Draw A Fish and watch it swim with the others", "url": "https://drawafish.com"}, {"title": "Gemini 2.5 Deep Think", "url": "https://blog.google/products/gemini/gemini-2-5-deep-think/"}, {"title": "FBI seized $40k from Linda Martin without charging her with a crime", "url": "https://reason.com/2025/07/28/the-fbi-took-her-40000-without-explaining-why-she-fought-back-and-lost/"}, {"title": "Fast (2019)", "url": "https://patrickcollison.com/fast"}, {"title": "Show HN: Pontoon \u2013 Open-source customer data syncs", "url": "https://github.com/pontoon-data/Pontoon"}]


1.output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now"):

->When asked a question that requires a tool, the ToolAgent performs the following steps internally:


->Sends the user query and the system prompt (containing the hn_tool definition) to the LLM.


->Receives the tool_call from the LLM (e.g., {"name": "fetch_top_hacker_news_stories", "arguments": {"top_n": 5}}).

->Parses the tool_call.

->Executes the fetch_top_hacker_news_stories function with top_n=5.


->Receives the result (the list of Hacker News stories).


->Appends this result as an "Observation" to the chat history.


->Makes a second call to the LLM with the updated history.


->Receives the final natural language response from the LLM, summarizing the Hacker News stories.

In [13]:
print(output)

The top 5 Hacker News stories right now are:

1. **Show HN: Draw A Fish and watch it swim with the others** - https://drawafish.com
2. **Gemini 2.5 Deep Think** - https://blog.google/products/gemini/gemini-2-5-deep-think/
3. **FBI seized $40k from Linda Martin without charging her with a crime** - https://reason.com/2025/07/28/the-fbi-took-her-40000-without-explaining-why-she-fought-back-and-lost/
4. **Fast (2019)** - https://patrickcollison.com/fast
5. **Show HN: Pontoon – Open-source customer data syncs** - https://github.com/pontoon-data/Pontoon
